This notebook aims to conduct feature engineering and final model building for our PD Model. Order of feature engineering is as followed:

1. Creation of Features
2. Feature Selection via IV <-> WoE binning selected features

This sequence is chosen due to the following reasons:

- Reduce chance of discarding features, which may only become useful after feature engineering
- Generating interaction features allow me to rank / select variables which are relevant and useful for PD Modeling
- Multicollinearity checks can be conducted once, instead of being repeated multiple times

# 0. Import Libraries


In [1]:
# === Standard libraries ===

import os
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# === WandB Logging  ===
import wandb

wandb.login(key=os.getenv("WANDB_API_KEY"))

# == Global Functions ==
from functions import *


# === Spark Session & Functions ===
from init_spark import start_spark

spark = start_spark()
from pyspark.sql.functions import (
    col,
    when,
    count,
    desc,
    isnan,
    isnull,
    lit,
    length,
    trim,
    lower,
    upper,
    to_date,
    concat_ws,
    regexp_extract,
    mean,
)
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DoubleType,
    IntegerType,
    DateType,
    NumericType,
    FloatType,
    LongType,
)


# === Pandas Dataframe & WoE Binning ===
import pandas as pd
from tabulate import tabulate
from optbinning import OptimalBinning
import numpy as np

# == Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

# === Machine Learning ===
from sklearn.linear_model import LogisticRegression as SkLogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    ConfusionMatrixDisplay,
)

# == Optbinning ==
from optbinning import OptimalPWBinning


# === Load Environment Variables ===
from dotenv import load_dotenv

load_dotenv()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/lunlun/.netrc
wandb: Currently logged in as: wlunlun1212 (wlunlun1212-singapore-management-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/lunlun/.netrc
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/16 16:14:35 WARN Utils: Your hostname, Chengs-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.77 instead (on interface en0)
25/0

4.0.0


True

In [2]:
# ============ Constants ======================

TARGET_COL = "default_status"
SAMPLE_FRAC = 0.2
SEED = 42
NOTEBOOK_RUN_NAME = "Feature Engineering"

IV_THRESHOLDS = {
    "useless (< 0.02)": 0.02,
    "weak (< 0.1)": 0.1,
    "medium (< 0.3)": 0.3,
    "strong (< 0.5)": 0.5,
}

# === Feature Categorisation ===
iv_categories = {
    "useless (< 0.02)": [],
    "weak (< 0.1)": [],
    "medium (< 0.3)": [],
    "strong (< 0.5)": [],
    "suspicious": [],
    "no_variation": [],
}

In [3]:
# ====================== Reusable Functions ======================


# == 0. Outliers ==
# == Outliers Handling ==
def compute_outlier_pct(df, col_name, lower_pct=0.25, upper_pct=0.75):
    """Computes pct of outliers per column based on IQR method"""

    # 1. Compute percentile bounds
    quantiles = df.approxQuantile(col_name, [lower_pct, upper_pct], 0.01)
    q1, q3 = quantiles[0], quantiles[1]
    iqr = q3 - q1

    # 2. Obtain lower and upper bound, any data points outside of this are seen as outliers
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    total_rows = df.count()

    return round(
        df.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound)).count()
        / total_rows
        * 100,
        2,
    )


def display_distributions(df):
    """Takes in Spark Dataframe. Samples it and display distribution for skewness checking"""
    # 1. Select numerical columns
    numeric_cols = [
        field.name for field in df.schema if isinstance(field.dataType, NumericType)
    ]

    # 2. Sample small portion of data (e.g., 5%) and convert to pandas
    sample_df = df.select(numeric_cols).sample(fraction=0.1, seed=42)
    sample_pdf = sample_df.toPandas()

    # 3. Plot histograms as subplots
    n_cols = 3  # Number of plots per row
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten()

    for i, col_name in enumerate(numeric_cols):
        axes[i].hist(sample_pdf[col_name].dropna(), bins=50, color="skyblue")
        axes[i].set_title(col_name, fontsize=10)
        axes[i].tick_params(axis="x", rotation=45)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


def inspect_outliers_iqr(df: DataFrame, columns: list, sample_size: int = 5):
    for col_name in columns:
        try:
            print(f"\n📊 Inspecting Outliers for Column: `{col_name}`")

            # Step 1: Calculate Q1, Q3, and IQR
            q1, q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr

            print(f"Q1 = {q1}, Q3 = {q3}, IQR = {iqr}")
            print(f"Lower Bound = {lower}, Upper Bound = {upper}")

            # Step 2: Count outliers
            outlier_count = df.filter(
                (col(col_name) < lower) | (col(col_name) > upper)
            ).count()
            total_count = df.count()
            outlier_pct = round(outlier_count / total_count * 100, 2)
            print(f"Outlier Count: {outlier_count} ({outlier_pct}%)")

            # Step 3: Sample outlier values (top and bottom)
            print(f"🔼 Top Outliers (>{upper}):")
            df.filter(col(col_name) > upper).select(col_name).orderBy(
                col(col_name).desc()
            ).show(sample_size)

            print(f"🔽 Bottom Outliers (<{lower}):")
            df.filter(col(col_name) < lower).select(col_name).orderBy(
                col(col_name)
            ).show(sample_size)

        except Exception as e:
            print(f"❌ Could not process column `{col_name}`: {str(e)}")


def winsorise_col(df, col_name, operator: str, condition_val, final_val):
    """
    Winsorises a column by replacing values above a certain condition with a final value.

    Args:
        df (DataFrame): The input DataFrame.
        col_name (str): The name of the column to winsorise.
        condition_val (float): The value above which to replace with final_val (cut-off)
        final_val (float): The value to replace with.

    Returns:
        DataFrame: The DataFrame with the winsorised column.
    """
    print("✅ Winsorising column:", col_name, "...")

    if operator == "<":
        return df.withColumn(
            col_name,
            when(col(col_name) < condition_val, final_val).otherwise(col(col_name)),
        )

    elif operator == ">":
        return df.withColumn(
            col_name,
            when(col(col_name) > condition_val, final_val).otherwise(col(col_name)),
        )


def retain_rows(
    df: DataFrame, col_name: str, condition_val: float, operator: str
) -> DataFrame:
    """
    Retains rows in the DataFrame where the specified column meets a condition.

    Returns:
        DataFrame: The DataFrame with the specified rows dropped.
    """

    if operator == "<=":
        return df.filter(col(col_name) <= condition_val)

    elif operator == "<":
        return df.filter(col(col_name) < condition_val)

    elif operator == ">":
        return df.filter(col(col_name) > condition_val)

    elif operator == ">=":
        return df.filter(col(col_name) >= condition_val)

    else:
        raise ValueError("Operator must be '>=' or '<='")


# == 1. Feature Engineering ==
def add_ratio_with_flag(df, numerator, denominator, ratio_col, flag_col, fill_value=-1):
    """
    Adds flag column to indicate invalid value upon division. Adds engineered feature column, and fills -1 upon invalid
    division operation.

    Parameters
    ----------
    df: str
        Spark Dataframe

    numerator: str
        Numerator Column Name
    denominator: str
        Denominator Column Name

    ratio_col: str
        Engineered Feature Name (Create Yourself)

    flag_col: str
        For e.g. annual_inc/loan_ratio_invalid

    fill_value: int
        Default: -1 (Placeholder in flag column, if invalid division operation)

    Returns
    -------
    Spark Dataframe with new columns (flag and divided column)
    """
    # ==  invalid denom (null or <= 0) -> 0 or -1 will be in flag columnis ==
    df = df.withColumn(
        flag_col,
        when(
            (col(denominator).isNull()) | (col(denominator) <= 0), 1
        ).otherwise(  # invalid → 1
            0
        ),  # valid → 0
    )

    # == safe ratio: real divide only when valid, else sentinel ==
    df = df.withColumn(
        ratio_col,
        when(col(flag_col) == 0, col(numerator) / col(denominator)).otherwise(
            lit(fill_value)
        ),
    )
    return df


# == 2. Feature Selection (WoE & IV) ==
def get_numerical_cols(df, target_col):
    """
    Get numerical columns of a Spark Dataframe and return list of numerical columns,
    excluding 'default_status'

    Parameters
    -----------

    df: Dataframe
        Spark Dataframe

    target_col: str
        default_status

    """
    return [
        f.name
        for f in df.schema.fields
        if isinstance(f.dataType, NumericType) and f.name != target_col
    ]


def get_categorical_cols(df, target_col):
    """
    Get categorical columns of a Spark Dataframe and return list of
    string columns

    Parameters
    -----------

    df: Dataframe
        Spark Dataframe

    target_col: str
        default_status

    """
    return [
        f.name
        for f in df.schema.fields
        if isinstance(f.dataType, StringType) and f.name != target_col
    ]


def classify_iv(iv_categories, feature, iv_score):
    """Adds 1 feature into respective IV categories of iv_categories dictionary"""
    for label, threshold in IV_THRESHOLDS.items():
        if iv_score < threshold:
            iv_categories[label].append((feature, iv_score))
            return

    iv_categories["suspicious"].append((feature, iv_score))


def bin_and_classify_feature(
    feature,
    x,
    y,
    dtype="numerical",
    monotonic_trend_type="auto",
    min_bin_size=0.05,
    max_n_bins=5,
    iv_categories=iv_categories,
):
    """
    Sample first before attempting to inspect binning & classifying into different IV Categories.
    Ensure only train_df (Pandas) fed in this function

    - Conducts Optimal Binning on a feature.
    - Outputs WoE table & Plot (to observe monotonic trend)
    - Categorises 1 feature into `iv_categories` dictionary

    Parameters
    ----------
    - feature: str
            Name of Feature
    - x: series
            e.g. train_df[feature]
    - y: series
            e.g. train_df['default_status']
    - dtype: str
            e.g. numerical (continuous /discrete / ordinal) & categorical (nominal) for OptBinning library
    - monotonic_trend_type: str
            e.g auto (default) / auto_asc_desc
    """

    # Fit binning with automatic solver
    optb = OptimalBinning(
        name=feature,
        dtype=dtype,
        monotonic_trend=monotonic_trend_type,
        min_bin_size=min_bin_size,  # min bin size is 1% (majority of our features have >= 1% outliers ... )
        solver="cp",
        max_n_bins=max_n_bins,
    )
    optb.fit(x, y)

    bin_table = optb.binning_table.build()

    #  Format binning table to display bins WoE & IV
    bin_df = pd.DataFrame(
        {
            "Bin": bin_table["Bin"].astype(str),
            "Count": round(bin_table["Count"], 4),
            "Default Rate (%)": round(bin_table["Event rate"], 4),
            "WOE": bin_table["WoE"],
            "IV": bin_table["IV"],
        }
    )

    # Get total IV for feature
    bin_df = bin_df[~bin_df["Bin"].str.contains("Total", na=False)]
    total_iv = bin_df.iloc[-1]["IV"]
    print(f"✅ Total IV for {feature}: {total_iv:.4f}")

    # Add feature to iv_categories
    classify_iv(iv_categories, feature, total_iv)

    # Print binning table
    print(tabulate(bin_df, headers="keys", tablefmt="fancy_grid", showindex=False))

    # Plot curve
    optb.binning_table.plot(metric="woe", figsize=(7, 4))


def tx_grade(df):
    """Takes in Pandas Dataframe and returns correct grade_numeric mapping from 'grade' column.
    This ensures ordinal natured of grade to be captured. Returns Pandas Dataframe."""
    # == Grade = Ordinal -> Dtype = `numerical` ==
    grade_map = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7}
    df["grade_numeric"] = df["grade"].map(grade_map)

    df = df.drop("grade", axis=1)

    return df


def get_updated_iv_categories(iv_categories, train_df):

    updated_iv_categories = {}
    for key, value in iv_categories.items():
        new_list = []
        if len(value) > 0 and key != "no_variation":
            for feature, iv_value in value:
                if feature in train_df.columns:
                    new_list.append((feature, iv_value))
            updated_iv_categories[key] = new_list
        else:
            continue

    return updated_iv_categories


def plot_iv_by_category(iv_categories):
    # === 1. Flatten into a clean DataFrame ===
    rows = []
    for category, items in iv_categories.items():
        for it in items:
            if isinstance(it, (tuple, list)) and len(it) >= 2:
                feature, iv = it[0], it[1]
                rows.append({"Feature": feature, "IV": float(iv), "Category": category})
            elif isinstance(it, str):
                rows.append({"Feature": it, "IV": 0.0, "Category": category})

    df_iv = pd.DataFrame(rows).sort_values(by="Feature")

    # === 2. Create faceted plots, showing only relevant features in each ===
    g = sns.catplot(
        data=df_iv,
        kind="bar",
        x="IV",
        y="Feature",
        col="Category",
        col_wrap=2,
        height=6,
        aspect=1.2,
        sharex=False,  # <== allow different x scales
        sharey=False,  # <== allow different features per facet
        palette="Set2",
    )

    g.figure.subplots_adjust(top=0.9, wspace=0.9)

    # === 3. Annotate bars ===
    for ax in g.axes.flat:
        for bar in ax.patches:
            width = bar.get_width()
            if width > 0:
                y = bar.get_y() + bar.get_height() / 2
                ax.text(width + 0.002, y, f"{width:.3f}", va="center", fontsize=8)

    # === 4. Final polish ===
    g.set_titles(col_template="{col_name}")
    g.set_xlabels("IV Score")
    g.set_ylabels("Feature")
    g.figure.suptitle("Information Value (IV) by Feature Category", fontsize=16)
    plt.show()
    return g


def peek_iv_score(updated_iv_categories, string_name, train_pdf):
    res_list = []
    for key, iv_list in updated_iv_categories.items():
        for feature, iv_score in iv_list:
            if (string_name in feature) and (feature in train_pdf.columns):
                res_list.append((feature, iv_score))

    return res_list


# == 3. WoE Transformation ==
def woe_bin_transform_train(df, non_mono_cols, target_col=TARGET_COL, monotonic="auto"):
    """
    Takes in train dataframe (Pandas)
    For each feature, separate Optbinning model is trained separately.
    If there are non-monotonic columns, use `auto_asc_desc` constraint in Optbinning

    Parameters
    -----------
    df: Pandas Dataframe
            train_pdf

    non_mono_cols: list
            non-monotonic columns

    target_col: str
            default_status (exists in train_pdf.columns)

    monotonic: str
            By default, monotonic='auto'
    """

    excluded_columns = ["id", "issue_d", "default_status", "earliest_cr_line"]
    loop_cols = [feature for feature in df.columns if feature not in excluded_columns]

    optb_dict = {}

    for column in loop_cols:
        # == Check if we should enforce monotonic trend on current feature ==
        if column in non_mono_cols:
            monotonic_trend = "auto_asc_desc"
        else:
            monotonic_trend = monotonic

        # == Determine dtype of current column ==
        if df[column].dtype == "object":
            dtype = "categorical"
        elif pd.api.types.is_numeric_dtype(df[column]):
            dtype = "numerical"
        else:
            continue  # skip unknown types

        optb = OptimalBinning(
            name=column,
            dtype=dtype,
            monotonic_trend=monotonic_trend,
            solver="cp",
            min_bin_size=0.05,
            max_n_bins=5,
        )

        try:
            optb.fit(df[column], df[target_col])
            df[column + "_woe"] = optb.transform(df[column].to_numpy(), metric="woe")
            df = df.drop(column, axis=1)
            optb_dict[column] = optb

        except Exception as e:
            print(f"❌ Error fitting {column}: {e}")

    return (df, optb_dict)


def apply_woe_transform(df, optb_dict):
    """
    Uses dictionary outputted by woe_bin_transform_train() to fit test dataframe,
    variable -> trainned Optbinning Model on train dataframe's feature.

    Transform each of test_pdf feature into `feature_woe`

    Parameters
    ----------
    df: Dataframe
            test_pdf

    optb_dict: dict
            Maps feature -> Optbinning Object
    """
    for column, optb in optb_dict.items():
        df[column + "_woe"] = optb.transform(df[column].to_numpy(), metric="woe")
        df = df.drop(columns=[column])
    return df


def pretty_print_iv_dict(iv_dict):
    for bucket, feats in iv_dict.items():
        print(f"\n{bucket}:")
        for feat, iv in feats:
            print(f"  {feat:<40} {float(iv):.6f}")

In [4]:
# == Remove all existing runs every time I run this notebook ==
NOTEBOOK_RUN_NAME = "PD Model Building II"
api = wandb.Api()
for run in api.runs(
    f"wlunlun1212-singapore-management-university/Credit Risk Modeling"
):
    if run.group == NOTEBOOK_RUN_NAME:
        run.delete()

In [5]:
df = spark.read.format("delta").load("../data/gold/medallion_cleaned_lc_data")

df.limit(10).toPandas()

25/08/16 16:14:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,credit_history_years,fico_score
0,87704389,1000.0,1000.0,36,10.49,32.50,B,2,MORTGAGE,39975.0,...,100.0,0.0,0.0,0.0,146014.0,60931.0,8300.0,70423.0,10,762.0
1,88862675,4800.0,4800.0,36,18.99,175.93,D,6,MORTGAGE,132000.0,...,78.6,0.0,0.0,0.0,314046.0,53279.0,500.0,54636.0,17,662.0
2,85636448,4075.0,4075.0,36,14.49,140.25,C,1,RENT,12000.0,...,100.0,0.0,0.0,0.0,27142.0,18772.0,5200.0,20342.0,4,752.0
3,88012886,23975.0,23975.0,60,15.59,577.82,C,2,RENT,89999.0,...,95.2,0.0,0.0,0.0,48652.0,14397.0,31500.0,12684.0,18,732.0
4,87989149,8000.0,8000.0,36,12.79,268.75,C,10,OWN,30000.0,...,100.0,20.0,1.0,0.0,55262.0,29807.0,11600.0,34562.0,9,707.0
5,85582885,5600.0,5600.0,36,12.79,188.13,C,10,MORTGAGE,109000.0,...,81.2,0.0,0.0,0.0,421855.0,50608.0,3000.0,58902.0,17,702.0
6,88494037,6000.0,6000.0,36,10.49,194.99,B,0,OWN,55000.0,...,100.0,0.0,0.0,0.0,27352.0,7421.0,19500.0,7852.0,15,807.0
7,88143505,24000.0,24000.0,60,15.59,578.42,C,10,MORTGAGE,65000.0,...,94.7,50.0,0.0,0.0,310036.0,38795.0,14000.0,28679.0,13,682.0
8,86216472,3200.0,3200.0,36,9.49,102.50,B,6,MORTGAGE,72000.0,...,94.3,0.0,0.0,0.0,112899.0,60713.0,4000.0,57816.0,12,747.0
9,87624578,3600.0,3600.0,36,8.59,113.80,A,0,RENT,70000.0,...,56.2,100.0,0.0,0.0,63980.0,40699.0,7500.0,54780.0,30,692.0


### 0.0 Base Model Performance


### 0.2 Skewness & Outlier Treatment

Before feature engineering, we will be dealing with outliers and skewed distributions, which can distort credit risk models. They can dominate learning, causing bias or overfitting in our PD Model. Let's first identify features which are highly skewed.


To ensure that our WoE & IV Feature Selection produce better binning (more balanced bins) and smoother WoE (if a feature separates good and bad outcomes properly), we should visualise our distributions to understand our dataset better.


From the original distribution of our cleaned dataset, we can observe that we have many skewed features, which is common is credit risk datasets. Before dealing with skewness, it is important to handle outliers first. Skewness measures the symmetry of a feature, and a few outliers can over-inflate the skewness of a feature, rendering the skewness value inaccurate. This can lead to us blindly applying transformations to a feature, when it does harm to our PD Model.

As such, based on the nature of an outlier, they will be dealt in different ways.

- **Clear Data Error**: Trim
- **Real Outlier but Rare (Domain Knowledge Based)**: Winsorise till IQR Bounds
- **Outlier is Real & Meaningful** : Let WoE binning handle this outlier


WoE binning is robust against outliers. There will be no skewness treatment for our Logistic Regression Model.


In [ ]:
def compute_outlier_pct(df, col_name, lower_pct=0.25, upper_pct=0.75):
    """Computes pct of outliers per column based on IQR method"""

    # 1. Compute percentile bounds
    quantiles = df.approxQuantile(col_name, [lower_pct, upper_pct], 0.01)
    q1, q3 = quantiles[0], quantiles[1]
    iqr = q3 - q1

    # 2. Obtain lower and upper bound, any data points outside of this are seen as outliers
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    total_rows = df.count()

    return round(
        df.filter((col(col_name) < lower_bound) | (col(col_name) > upper_bound)).count()
        / total_rows
        * 100,
        2,
    )


def display_distributions(df):
    """Takes in Spark Dataframe. Samples it and display distribution for skewness checking"""
    # 1. Select numerical columns
    numeric_cols = [
        field.name for field in df.schema if isinstance(field.dataType, NumericType)
    ]

    # 2. Sample small portion of data (e.g., 5%) and convert to pandas
    sample_df = df.select(numeric_cols).sample(fraction=0.1, seed=42)
    sample_pdf = sample_df.toPandas()

    # 3. Plot histograms as subplots
    n_cols = 3  # Number of plots per row
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten()

    for i, col_name in enumerate(numeric_cols):
        axes[i].hist(sample_pdf[col_name].dropna(), bins=50, color="skyblue")
        axes[i].set_title(col_name, fontsize=10)
        axes[i].tick_params(axis="x", rotation=45)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


def inspect_outliers_iqr(df: DataFrame, columns: list, sample_size: int = 5):
    for col_name in columns:
        try:
            print(f"\n📊 Inspecting Outliers for Column: `{col_name}`")

            # Step 1: Calculate Q1, Q3, and IQR
            q1, q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr

            print(f"Q1 = {q1}, Q3 = {q3}, IQR = {iqr}")
            print(f"Lower Bound = {lower}, Upper Bound = {upper}")

            # Step 2: Count outliers
            outlier_count = df.filter(
                (col(col_name) < lower) | (col(col_name) > upper)
            ).count()
            total_count = df.count()
            outlier_pct = round(outlier_count / total_count * 100, 2)
            print(f"Outlier Count: {outlier_count} ({outlier_pct}%)")

            # Step 3: Sample outlier values (top and bottom)
            print(f"🔼 Top Outliers (>{upper}):")
            df.filter(col(col_name) > upper).select(col_name).orderBy(
                col(col_name).desc()
            ).show(sample_size)

            print(f"🔽 Bottom Outliers (<{lower}):")
            df.filter(col(col_name) < lower).select(col_name).orderBy(
                col(col_name)
            ).show(sample_size)

        except Exception as e:
            print(f"❌ Could not process column `{col_name}`: {str(e)}")


def winsorise_col(df, col_name, operator: str, condition_val, final_val):
    """
    Winsorises a column by replacing values above a certain condition with a final value.

    Args:
        df (DataFrame): The input DataFrame.
        col_name (str): The name of the column to winsorise.
        condition_val (float): The value above which to replace with final_val (cut-off)
        final_val (float): The value to replace with.

    Returns:
        DataFrame: The DataFrame with the winsorised column.
    """
    print("✅ Winsorising column:", col_name, "...")

    if operator == "<":
        return df.withColumn(
            col_name,
            when(col(col_name) < condition_val, final_val).otherwise(col(col_name)),
        )

    elif operator == ">":
        return df.withColumn(
            col_name,
            when(col(col_name) > condition_val, final_val).otherwise(col(col_name)),
        )


def retain_rows(
    df: DataFrame, col_name: str, condition_val: float, operator: str
) -> DataFrame:
    """
    Retains rows in the DataFrame where the specified column meets a condition.

    Returns:
        DataFrame: The DataFrame with the specified rows dropped.
    """

    if operator == "<=":
        return df.filter(col(col_name) <= condition_val)

    elif operator == "<":
        return df.filter(col(col_name) < condition_val)

    elif operator == ">":
        return df.filter(col(col_name) > condition_val)

    elif operator == ">=":
        return df.filter(col(col_name) >= condition_val)

    else:
        raise ValueError("Operator must be '>=' or '<='")


# == PD Outlier Treatment ==

# == 1. Find outliers % per column, without sampling (may risk missing absurd placeholders) ==

numeric_df5 = df.select(
    [feature.name for feature in df.schema if isinstance(feature.dataType, NumericType)]
)

outliers_dict = {}

for feature in numeric_df5.schema.fields:
    col_name = feature.name
    data_type = feature.dataType

    if isinstance(data_type, NumericType):
        outlier_pct = compute_outlier_pct(numeric_df5, col_name)
        if outlier_pct > 0:
            outliers_dict[col_name] = outlier_pct


print("❌ Outlier Percentage by Feature (sorted):")
for k, v in sorted(outliers_dict.items(), key=lambda item: item[1], reverse=True):
    print(f"{k}: {round(v, 2)}%")

From the original distribution of our cleaned dataset, we can observe that we have many skewed features, which is common is credit risk datasets. Before dealing with skewness, it is important to handle outliers first. Skewness measures the symmetry of a feature, and a few outliers can over-inflate the skewness of a feature, rendering the skewness value inaccurate. This can lead to us blindly applying transformations to a feature, when it does harm to our PD Model.

As such, based on the nature of an outlier, they will be dealt in different ways.

- **Clear Data Error**: Trim
- **Real Outlier but Rare (Domain Knowledge Based)**: Winsorise till IQR Bounds
- **Outlier is Real & Meaningful** : Let WoE binning handle this outlier


In [ ]:
# == 1. Get Numeric Columns from Spark Dataframe ==
numerical_cols1 = get_numerical_cols(df1, TARGET_COL)

# == 2. Obtain Sampled Pandas Train, Test Dataframe from Spark Dataframe, orders them ==
train_pdf1, test_pdf1 = sample_split_order(
    initial_df=df1, sample_frac=0.1, cut_off_date=CUT_DATE, date_col="issue_d"
)

# == 3. Observe IV of Numerical Variables ==
for feature in numerical_cols1:
    if feature == "id":
        continue

    print(f"\n🔍 Feature: {feature}")

    if train_pdf1[feature].nunique() < 2:
        print("⚠️ Not enough variation. Skipping.")
        iv_categories["no_variation"].append(feature)
        continue

    x, y = train_pdf1[feature], train_pdf1[TARGET_COL]

    bin_and_classify_feature(feature=feature, x=x, y=y, monotonic_trend_type="auto")

# == Include grade_numeric as well (ordinal feature) ==
train_pdf1 = tx_grade(train_pdf1)

bin_and_classify_feature(
    "grade_numeric",
    train_pdf1["grade_numeric"],
    train_pdf1[TARGET_COL],
    dtype="numerical",
    monotonic_trend_type="auto",
)

### 2.2 Categorical Features IV


In [ ]:
# == Observe IV of Categorical Variables ==
categorical_features = get_categorical_cols(df1, TARGET_COL)

for feat in categorical_features:
    if feat != "grade":
        bin_and_classify_feature(
            feat,
            train_pdf1[feat],
            train_pdf1[TARGET_COL],
            dtype="categorical",
            monotonic_trend_type="auto",
        )

### 2.3 Feature Importance Rankings


In [ ]:
# == Compiled Feature Rankings (Categorical & Numerical) ==
plot_iv_by_category(iv_categories)

### 2.4 Feature Filtering

Now that we have identified the more relevant/important features for PD Modeling, lets drop the features and observe feature importance ranking & WoE trends.


In [ ]:
# == Define columns to abandon & keep (Based on train_pdf1) ==
no_variation_columns = [
    item
    for category, feature_list in iv_categories.items()
    if category == "no_variation"
    for item in feature_list
]

useless_cols = [
    feature
    for category, feature_list in iv_categories.items()
    if category == "useless (< 0.02)"
    for feature, _ in feature_list
]

throw_cols = no_variation_columns + useless_cols
keep_cols = [c for c in train_pdf1.columns if c not in throw_cols]

train_pdf1 = train_pdf1.drop(columns=throw_cols)
print("✅ Dropped columns with IV < 0.02 and columns with no variation in TRAIN_DF1")

In [ ]:
# == Leftover Columns IV in TRAIN_DF1 ==
updated_iv_categories = get_updated_iv_categories(iv_categories, train_pdf1)

plot_iv_by_category(updated_iv_categories)

In [ ]:
train_pdf1.columns

In [ ]:
# == Observe WoE & IV of train_pdf1 ==
for feature in sorted(train_pdf1.columns):
    excluded_columns = ["id", "issue_d", "default_status"]
    x, y = train_pdf1[feature], train_pdf1[TARGET_COL]

    if feature in excluded_columns:
        continue

    if train_pdf1[feature].dtype == "object":
        bin_and_classify_feature(
            feature=feature,
            x=x,
            y=y,
            monotonic_trend_type="auto",
            dtype="categorical",
            max_n_bins=5,
        )
    else:
        bin_and_classify_feature(
            feature=feature, x=x, y=y, monotonic_trend_type="auto", max_n_bins=5
        )

In [ ]:
# == Inspect Non Monotonic Features to handle later on ==

non_monotonic_features = ["tot_cur_bal", "tot_hi_cred_lim", "fico_score/inq_last_6mths"]

for feature in non_monotonic_features:
    bin_and_classify_feature(feature, x=train_pdf1[feature], y=train_pdf1[TARGET_COL])

For these non-monotonic features, we shall attempt to enforce a strict monotonic trend constraint on WoE. If IV drops severely, the feature shall be dropped.


In [ ]:
# == Inspect forced WoE monotonicity constraint ==
for non_mono in ["tot_cur_bal", "tot_hi_cred_lim", "fico_score/inq_last_6mths"]:
    bin_and_classify_feature(
        non_mono,
        train_pdf1[non_mono],
        train_pdf1[TARGET_COL],
        monotonic_trend_type="auto_asc_desc",
    )

As seen, enforcing monotonicity constraint caused `fico_score/inq_last_6mths` resulted in a drop of 0.04 in IV. This variable should be dropped, since monotonicity constraint is important for Logistic Regression.


In [ ]:
train_pdf1 = train_pdf1.drop("fico_score/inq_last_6mths", axis=1)
train_pdf1 = train_pdf1.drop("fico_score/inq_last_6mths_invalid", axis=1)

assert ("fico_score/inq_last_6mths" not in train_pdf1.columns) and (
    "fico_score/inq_last_6mths_invalid" not in train_pdf1.columns
)

In [ ]:
updated_iv_categories = get_updated_iv_categories(updated_iv_categories, train_pdf1)
plot_iv_by_category(updated_iv_categories)

In [ ]:
# == Before WoE Binning Transformation ==
train_pdf1.head()

In [ ]:
# == Ensure Same Columns between train_df and test_df (Drop `grade`) ==

# == 1. Add grade_numeric, drop grade column
test_pdf1 = tx_grade(test_pdf1)

# = 2. Select only columns train_pdf1 has ==
test_pdf1 = test_pdf1[train_pdf1.columns]
test_pdf1.head()

In [ ]:
# == Check column discrepancies ==
set(test_pdf1.columns) - set(train_pdf1.columns)

In [ ]:
# == 1. WoE binning Transformation on train_df ==
non_monotonic_cols = ["tot_cur_bal", "tot_hi_cred_lim"]

# == 2. Transform Train Dataset ==
train_pdf1, optb_dict = woe_bin_transform_train(
    train_pdf1, non_monotonic_features, TARGET_COL
)

train_pdf1.head()

In [ ]:
# == Transform Test Dataset ==
test_pdf1 = apply_woe_transform(test_pdf1, optb_dict)
test_pdf1.head()

In [ ]:
# == Evaluate Model Performance upon WoE Transformation ==
run_model_checkpoint(
    train_pdf1,
    test_pdf1,
    run_name="log_reg_woe_transformation",
    model_type="Logistic Regression",
    run_group=NOTEBOOK_RUN_NAME,
)

In [ ]:
train_pdf2 = train_pdf1
test_pdf2 = test_pdf1

# 3. A/B Testing (Interaction Feature Selection)


In [ ]:
# = Find base vs interaction features ==
excluded_columns = ["id", "default_status", "issue_d"]
base_feats = sorted(
    [
        col
        for col in train_pdf2.columns
        if "_x_" not in col
        and "/" not in col
        and col != "avail_ratio_woe"
        and col not in excluded_columns
    ]
)

interaction_features = sorted(
    [
        feature
        for feature in train_pdf2.columns
        if feature not in base_feats and feature not in excluded_columns
    ]
)


interaction_features

In [ ]:
from sklearn.metrics import roc_auc_score

METRIC = "gini"  # "auc" or "gini"
THRESH_GINI = 0.0015  # same but for Gini


# == WoE transformed variables are already scaled in log odds ==
def score(train_df, test_df, features):
    X_tr = train_df[features]
    y_tr = train_df[TARGET_COL]
    X_te = test_df[features]
    y_te = test_df[TARGET_COL]
    model = LogisticRegression(solver="lbfgs", C=1.0, max_iter=1000, penalty="l2")
    model.fit(X_tr, y_tr)
    auc = roc_auc_score(y_te, model.predict_proba(X_te)[:, 1])
    return 2 * auc - 1


def pick_interactions(train_df, test_df, base_features, interaction_features):
    base_score = score(train_df, test_df, base_features)
    print(f"Base {METRIC.upper()}: {base_score:.4f}")
    keep = []
    for f in interaction_features:
        s = score(train_df, test_df, base_features + [f])
        delta = s - base_score
        need = THRESH_GINI
        print(f"{f:40s} -> {METRIC.upper()}: {s:.4f}  Δ={delta:+.4f}")
        if delta >= need:
            keep.append(f)

    return base_features + keep


# ==== example usage ====


final_feats = pick_interactions(train_pdf2, test_pdf2, base_feats, interaction_features)

In [ ]:
sorted(final_feats)

In [ ]:
train_pdf2 = train_pdf2[final_feats + excluded_columns]

In [ ]:
run_model_checkpoint(
    train_pdf2, test_pdf2, "log_reg_ab_test", "Logistic Regression", NOTEBOOK_RUN_NAME
)

In [ ]:
# == Obtain IV scores of each feature currently in train_pdf2 ==
updated_iv_categories_woe = {
    bucket: [
        (f"{feat}_woe", iv) for feat, iv in feats if f"{feat}_woe" in train_pdf2.columns
    ]
    for bucket, feats in updated_iv_categories.items()
}

plot_iv_by_category(updated_iv_categories_woe)